# Named Entity Recognition using LSTM

### 1. Preparing vocabulary for Word Embeddings

In [2]:
import pandas as pd
import numpy as np
import nltk
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
df = pd.read_csv('./data/ner_dataset.csv')

In [4]:
words = df['Word'].str.lower()
words = words.drop_duplicates().values
words = np.insert(words, 0, 'EMPT')  #For padded sequences
print(words)
print(words.shape)

['EMPT' 'thousands' 'of' ..., 'indicative' '3700' 'bermel']
(31805,)


In [5]:
word_indices = {}

In [6]:
def map_word_indices(words):
    for i in range(len(words)):
        word_indices[words[i]] = i

In [7]:
map_word_indices(words)
print(word_indices)

{'EMPT': 0, 'thousands': 1, 'of': 2, 'demonstrators': 3, 'have': 4, 'marched': 5, 'through': 6, 'london': 7, 'to': 8, 'protest': 9, 'the': 10, 'war': 11, 'in': 12, 'iraq': 13, 'and': 14, 'demand': 15, 'withdrawal': 16, 'british': 17, 'troops': 18, 'from': 19, 'that': 20, 'country': 21, '.': 22, 'families': 23, 'soldiers': 24, 'killed': 25, 'conflict': 26, 'joined': 27, 'protesters': 28, 'who': 29, 'carried': 30, 'banners': 31, 'with': 32, 'such': 33, 'slogans': 34, 'as': 35, '"': 36, 'bush': 37, 'number': 38, 'one': 39, 'terrorist': 40, 'stop': 41, 'bombings': 42, 'they': 43, 'houses': 44, 'parliament': 45, 'a': 46, 'rally': 47, 'hyde': 48, 'park': 49, 'police': 50, 'put': 51, 'marchers': 52, 'at': 53, '10,000': 54, 'while': 55, 'organizers': 56, 'claimed': 57, 'it': 58, 'was': 59, '1,00,000': 60, 'comes': 61, 'on': 62, 'eve': 63, 'annual': 64, 'conference': 65, 'britain': 66, "'s": 67, 'ruling': 68, 'labor': 69, 'party': 70, 'southern': 71, 'english': 72, 'seaside': 73, 'resort': 74, 

In [8]:
def get_index_matrix(s):
    id_matrix = []
    w = nltk.word_tokenize(s)
    w = [i.lower() for i in w]
    for i in w:
        if i in words:
            id_matrix.append(word_indices[i])
        else :
            id_matrix.append(31805) #Unknown token
    return id_matrix

In [9]:
sentence = "My name is Edward."
print(get_index_matrix(sentence))

[1397, 2961, 76, 10086, 22]


Generating this ID matrix will be useful for training the embedding layer.

### 2. Preparing Data-set

In [10]:
df.head()

,Sentence,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [11]:
tags = df['Tag'].drop_duplicates().values
print(tags)
print("Number of classes : " + str(len(tags)))

['O' 'B-geo' 'B-gpe' 'B-per' 'I-geo' 'B-org' 'I-org' 'B-tim' 'B-art'
 'I-art' 'I-per' 'I-gpe' 'I-tim' 'B-nat' 'B-eve' 'I-eve' 'I-nat']
Number of classes : 17


In [12]:
one_hot_tags = pd.get_dummies(df['Tag']).values
print(pd.get_dummies(df['Tag']))

         B-art  B-eve  B-geo  B-gpe  B-nat  B-org  B-per  B-tim  I-art  I-eve  \
0            0      0      0      0      0      0      0      0      0      0   
1            0      0      0      0      0      0      0      0      0      0   
2            0      0      0      0      0      0      0      0      0      0   
3            0      0      0      0      0      0      0      0      0      0   
4            0      0      0      0      0      0      0      0      0      0   
5            0      0      0      0      0      0      0      0      0      0   
6            0      0      1      0      0      0      0      0      0      0   
7            0      0      0      0      0      0      0      0      0      0   
8            0      0      0      0      0      0      0      0      0      0   
9            0      0      0      0      0      0      0      0      0      0   
10           0      0      0      0      0      0      0      0      0      0   
11           0      0      0

In [13]:
x_data = []
y_data = []
temp_y = []
s = ''
for index,rows in df.iterrows():
    s = s + rows['Word'].lower() + ' '
    temp_y.append(one_hot_tags[index])
    if rows['Word'] == ".":
        x_data.append(get_index_matrix(s))
        y_data.append(temp_y)
        temp_y = []
        s = ''
        print('Sentences processed : ' + str(len(x_data)))

Sentences processed : 1
Sentences processed : 2
Sentences processed : 3
Sentences processed : 4
Sentences processed : 5
Sentences processed : 6
Sentences processed : 7
Sentences processed : 8
Sentences processed : 9
Sentences processed : 10
Sentences processed : 11
Sentences processed : 12
Sentences processed : 13
Sentences processed : 14
Sentences processed : 15
Sentences processed : 16
Sentences processed : 17
Sentences processed : 18
Sentences processed : 19
Sentences processed : 20
Sentences processed : 21
Sentences processed : 22
Sentences processed : 23
Sentences processed : 24
Sentences processed : 25
Sentences processed : 26
Sentences processed : 27
Sentences processed : 28
Sentences processed : 29
Sentences processed : 30
Sentences processed : 31
Sentences processed : 32
Sentences processed : 33
Sentences processed : 34
Sentences processed : 35
Sentences processed : 36
Sentences processed : 37
Sentences processed : 38
Sentences processed : 39
Sentences processed : 40
Sentences

Sentences processed : 325
Sentences processed : 326
Sentences processed : 327
Sentences processed : 328
Sentences processed : 329
Sentences processed : 330
Sentences processed : 331
Sentences processed : 332
Sentences processed : 333
Sentences processed : 334
Sentences processed : 335
Sentences processed : 336
Sentences processed : 337
Sentences processed : 338
Sentences processed : 339
Sentences processed : 340
Sentences processed : 341
Sentences processed : 342
Sentences processed : 343
Sentences processed : 344
Sentences processed : 345
Sentences processed : 346
Sentences processed : 347
Sentences processed : 348
Sentences processed : 349
Sentences processed : 350
Sentences processed : 351
Sentences processed : 352
Sentences processed : 353
Sentences processed : 354
Sentences processed : 355
Sentences processed : 356
Sentences processed : 357
Sentences processed : 358
Sentences processed : 359
Sentences processed : 360
Sentences processed : 361
Sentences processed : 362
Sentences pr

Sentences processed : 645
Sentences processed : 646
Sentences processed : 647
Sentences processed : 648
Sentences processed : 649
Sentences processed : 650
Sentences processed : 651
Sentences processed : 652
Sentences processed : 653
Sentences processed : 654
Sentences processed : 655
Sentences processed : 656
Sentences processed : 657
Sentences processed : 658
Sentences processed : 659
Sentences processed : 660
Sentences processed : 661
Sentences processed : 662
Sentences processed : 663
Sentences processed : 664
Sentences processed : 665
Sentences processed : 666
Sentences processed : 667
Sentences processed : 668
Sentences processed : 669
Sentences processed : 670
Sentences processed : 671
Sentences processed : 672
Sentences processed : 673
Sentences processed : 674
Sentences processed : 675
Sentences processed : 676
Sentences processed : 677
Sentences processed : 678
Sentences processed : 679
Sentences processed : 680
Sentences processed : 681
Sentences processed : 682
Sentences pr

Sentences processed : 985
Sentences processed : 986
Sentences processed : 987
Sentences processed : 988
Sentences processed : 989
Sentences processed : 990
Sentences processed : 991
Sentences processed : 992
Sentences processed : 993
Sentences processed : 994
Sentences processed : 995
Sentences processed : 996
Sentences processed : 997
Sentences processed : 998
Sentences processed : 999
Sentences processed : 1000
Sentences processed : 1001
Sentences processed : 1002
Sentences processed : 1003
Sentences processed : 1004
Sentences processed : 1005
Sentences processed : 1006
Sentences processed : 1007
Sentences processed : 1008
Sentences processed : 1009
Sentences processed : 1010
Sentences processed : 1011
Sentences processed : 1012
Sentences processed : 1013
Sentences processed : 1014
Sentences processed : 1015
Sentences processed : 1016
Sentences processed : 1017
Sentences processed : 1018
Sentences processed : 1019
Sentences processed : 1020
Sentences processed : 1021
Sentences proces

Sentences processed : 1359
Sentences processed : 1360
Sentences processed : 1361
Sentences processed : 1362
Sentences processed : 1363
Sentences processed : 1364
Sentences processed : 1365
Sentences processed : 1366
Sentences processed : 1367
Sentences processed : 1368
Sentences processed : 1369
Sentences processed : 1370
Sentences processed : 1371
Sentences processed : 1372
Sentences processed : 1373
Sentences processed : 1374
Sentences processed : 1375
Sentences processed : 1376
Sentences processed : 1377
Sentences processed : 1378
Sentences processed : 1379
Sentences processed : 1380
Sentences processed : 1381
Sentences processed : 1382
Sentences processed : 1383
Sentences processed : 1384
Sentences processed : 1385
Sentences processed : 1386
Sentences processed : 1387
Sentences processed : 1388
Sentences processed : 1389
Sentences processed : 1390
Sentences processed : 1391
Sentences processed : 1392
Sentences processed : 1393
Sentences processed : 1394
Sentences processed : 1395
S

...

Sentences processed : 46342
Sentences processed : 46343
Sentences processed : 46344
Sentences processed : 46345
Sentences processed : 46346
Sentences processed : 46347
Sentences processed : 46348
Sentences processed : 46349
Sentences processed : 46350
Sentences processed : 46351
Sentences processed : 46352
Sentences processed : 46353
Sentences processed : 46354
Sentences processed : 46355
Sentences processed : 46356
Sentences processed : 46357
Sentences processed : 46358
Sentences processed : 46359
Sentences processed : 46360
Sentences processed : 46361
Sentences processed : 46362
Sentences processed : 46363
Sentences processed : 46364
Sentences processed : 46365
Sentences processed : 46366
Sentences processed : 46367
Sentences processed : 46368
Sentences processed : 46369
Sentences processed : 46370
Sentences processed : 46371
Sentences processed : 46372
Sentences processed : 46373
Sentences processed : 46374
Sentences processed : 46375
Sentences processed : 46376
Sentences processed 

Sentences processed : 46701
Sentences processed : 46702
Sentences processed : 46703
Sentences processed : 46704
Sentences processed : 46705
Sentences processed : 46706
Sentences processed : 46707
Sentences processed : 46708
Sentences processed : 46709
Sentences processed : 46710
Sentences processed : 46711
Sentences processed : 46712
Sentences processed : 46713
Sentences processed : 46714
Sentences processed : 46715
Sentences processed : 46716
Sentences processed : 46717
Sentences processed : 46718
Sentences processed : 46719
Sentences processed : 46720
Sentences processed : 46721
Sentences processed : 46722
Sentences processed : 46723
Sentences processed : 46724
Sentences processed : 46725
Sentences processed : 46726
Sentences processed : 46727
Sentences processed : 46728
Sentences processed : 46729
Sentences processed : 46730
Sentences processed : 46731
Sentences processed : 46732
Sentences processed : 46733
Sentences processed : 46734
Sentences processed : 46735
Sentences processed 

Sentences processed : 47127
Sentences processed : 47128
Sentences processed : 47129
Sentences processed : 47130
Sentences processed : 47131
Sentences processed : 47132
Sentences processed : 47133
Sentences processed : 47134
Sentences processed : 47135
Sentences processed : 47136
Sentences processed : 47137
Sentences processed : 47138
Sentences processed : 47139
Sentences processed : 47140
Sentences processed : 47141
Sentences processed : 47142
Sentences processed : 47143
Sentences processed : 47144
Sentences processed : 47145
Sentences processed : 47146
Sentences processed : 47147
Sentences processed : 47148
Sentences processed : 47149
Sentences processed : 47150
Sentences processed : 47151
Sentences processed : 47152
Sentences processed : 47153
Sentences processed : 47154
Sentences processed : 47155
Sentences processed : 47156
Sentences processed : 47157
Sentences processed : 47158
Sentences processed : 47159
Sentences processed : 47160
Sentences processed : 47161
Sentences processed 

Sentences processed : 47431
Sentences processed : 47432
Sentences processed : 47433
Sentences processed : 47434
Sentences processed : 47435
Sentences processed : 47436
Sentences processed : 47437
Sentences processed : 47438
Sentences processed : 47439
Sentences processed : 47440
Sentences processed : 47441
Sentences processed : 47442
Sentences processed : 47443
Sentences processed : 47444
Sentences processed : 47445
Sentences processed : 47446
Sentences processed : 47447
Sentences processed : 47448
Sentences processed : 47449
Sentences processed : 47450
Sentences processed : 47451
Sentences processed : 47452
Sentences processed : 47453
Sentences processed : 47454
Sentences processed : 47455
Sentences processed : 47456
Sentences processed : 47457
Sentences processed : 47458
Sentences processed : 47459
Sentences processed : 47460
Sentences processed : 47461
Sentences processed : 47462
Sentences processed : 47463
Sentences processed : 47464
Sentences processed : 47465
Sentences processed 

Sentences processed : 47724
Sentences processed : 47725
Sentences processed : 47726
Sentences processed : 47727
Sentences processed : 47728
Sentences processed : 47729
Sentences processed : 47730
Sentences processed : 47731
Sentences processed : 47732
Sentences processed : 47733
Sentences processed : 47734
Sentences processed : 47735
Sentences processed : 47736
Sentences processed : 47737
Sentences processed : 47738
Sentences processed : 47739
Sentences processed : 47740
Sentences processed : 47741
Sentences processed : 47742
Sentences processed : 47743
Sentences processed : 47744
Sentences processed : 47745
Sentences processed : 47746
Sentences processed : 47747
Sentences processed : 47748
Sentences processed : 47749
Sentences processed : 47750
Sentences processed : 47751
Sentences processed : 47752
Sentences processed : 47753
Sentences processed : 47754
Sentences processed : 47755
Sentences processed : 47756
Sentences processed : 47757
Sentences processed : 47758
Sentences processed 

In [17]:
x_back = x_data
y_back = y_data

In [18]:
def pad_output(x,y):
    for i in range(len(x)):
        diff = len(x[i])-len(y[i])
        for j in range(diff):
            y[i] = np.insert(y[i],0,np.zeros(17),axis=0)
    return y
x_data = pad_sequences(x_data, maxlen=25)

In [21]:
y_data = pad_output(x_data,y_data)

Now, we have padded and indexed all 47,761 sentences. let's verify their format

In [22]:
print(x_data[23])
for i in y_data[23]:
    print(i)

[  4 300  12  10 310 311  14 312 302  91 246  42 313 114  25  53 279 314
 315  14 316 289 290 317  22]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

For ever word in a a sentence we have a one-hot encoded class it belongs to.

### 3. Making the model

In [23]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM

In [ ]:
model = Sequential()
model.add(Embedding(31805,150,input_length=25,dropout=0.3))
model.add(LSTM(17,activation='softmax',dropout_U=0.2, dropout_W=0.2,return_sequences=True))

C:\Users\mayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
C:\Users\mayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(17, activation="softmax", return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
x_train = x_data[:42985]

In [ ]:
print(x_data[1])